## Обработка курьеров

In [ ]:
import pandas as pd
import numpy as np
# pd.set_option('display.max_columns', None) 
# pd.set_option('display.width', 1000)     

In [2]:
df = pd.read_csv('data_1/10-01.csv')
df

,id_courier,date_group,courier_type,status,pools_count,order_count,shift_num,shift_count_per_day,delivery_time_shift,count_pool_shift,...,count_shift,order_counts,order_weight,is_back,hour,minute,weekday,time_since_last_tt,orders_last_hour,pools_last_hour
0,65709895,2024-10-04 09:45:00,Авто,Готов взять заказ,1,1,1.0,1,0.122222,1.0,...,1.0,1.0,0.671333,0,9,45,4,0.0,0.0,0.0
1,65709895,2024-10-04 10:00:00,Авто,Не работает,1,1,1.0,1,0.122222,1.0,...,1.0,1.0,0.671333,0,10,0,4,15.0,0.0,0.0
2,65709895,2024-10-04 10:15:00,Авто,Не работает,1,1,1.0,1,0.122222,1.0,...,1.0,1.0,0.671333,0,10,15,4,15.0,0.0,0.0
3,65709895,2024-10-04 10:30:00,Авто,Не работает,1,1,1.0,1,0.122222,1.0,...,1.0,1.0,0.671333,0,10,30,4,15.0,0.0,0.0
4,65709895,2024-10-04 10:45:00,Авто,Не работает,1,1,1.0,1,0.122222,1.0,...,1.0,1.0,0.671333,0,10,45,4,15.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28378891,1799997361,2025-01-31 20:00:00,Авто,Не работает,2,6,1.0,1,0.616667,1.0,...,1.0,3.0,11.773000,0,20,0,4,15.0,0.0,0.0
28378892,1799997361,2025-01-31 20:15:00,Авто,Не работает,2,6,1.0,1,0.616667,1.0,...,1.0,3.0,11.773000,0,20,15,4,15.0,0.0,0.0
28378893,1799997361,2025-01-31 20:30:00,Авто,Не работает,2,6,1.0,1,0.616667,1.0,...,1.0,3.0,11.773000,0,20,30,4,15.0,0.0,0.0
28378894,1799997361,2025-01-31 20:45:00,Авто,Не работает,2,6,1.0,1,0.616667,1.0,...,1.0,3.0,11.773000,0,20,45,4,15.0,0.0,0.0


### Скачиваем данные и обрезаем для удобства анализа

In [2]:
# ПОМЕНЯТЬ ДАТУ
date_split = '2025-02-01'
file_path = 'data/12-01.csv'

In [3]:
df = pd.read_csv(file_path)

# Преобразование временных столбцов в datetime
date_columns = [
    'date_status', 'first_date_order', 'last_date_order', 'date_supply', 'date_supply_untill', 'date_collected', 'date_delivery_start', 'date_delivered', 'pool_date_start']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')
    # Округление до минут (обнуляем секунды и микросекунды)
    df[col] = df[col].dt.floor('min')  # Используем 'min' вместо 'T'

# Обрежем файл для удобства обработки (НЕЗАБЫТЬ УБРАТЬ ЭТО УСЛОВИЕ)
df['date'] = df['pool_date_start'].dt.date
# df=df[df['date']<='2025-01-20']


In [4]:
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
df.sort_values('date_delivery_start',inplace=True)
# example=df[(df['id_courier']==94477626) &  (df['date']=='2024-01-05') ]
# example

### Фильтрация данных и очистка от пропусков 

In [5]:
df = df[df['date_delivered'] > df['date_delivery_start']]
df = df[df['date_status'] > df['first_date_order']]

df['courier_type']=df['courier_type'].fillna('Тип курьера неизвестен')
df.dropna(subset=['id_courier'], inplace=True)
df.drop(columns=['id_courier.1'], inplace=True)

nan_counts = df.isna().sum()
print(nan_counts)

# Фильтрация по источникам
vv_source = [21, 1, 15, 78, 79, 67, 80, 71, 33, 2, 5, 3, 4, 39, 66, 58, 63]
df = df[df['id_source'].isin(vv_source)]

df=df[(~df['date_delivery_start'].isna())&(~df['date_delivered'].isna())]
df['total_order_weight'] = df['total_order_weight'].fillna(0)

df.drop(columns=['distance','pool_date_start'],inplace=True)
nan_counts = df.isna().sum()
print(nan_counts)


id_pool                     0
id_courier                  0
courier_type                0
id_job                      0
pool_date_start         29649
sort_order                  0
id_general                  0
distance               738264
status_str                  0
date_status                 0
id_general.1                0
total_order_weight         47
total_orders                0
first_date_order            0
last_date_order             0
date_supply                 0
date_supply_untill          0
date_collected          61025
date_delivery_start         0
date_delivered              0
id_service                  0
id_source                   0
latitude_order              0
longitude_order             0
N                           0
shirota_tt                  0
dolgota_tt                  0
gen_distance           176391
date                    29649
dtype: int64
id_pool                     0
id_courier                  0
courier_type                0
id_job                     

Сейчас мы не используем типы сервисов, поэтому этот блок можно не запускать 


In [ ]:
# import pandas as pd

# # Читаем сервисы
# service = pd.read_excel('../data/Сервисы.xlsx')

# # Группы сервисов
# service_groups = {
#     'Доставка сейчас (2 часа)': {'Доставка сейчас', 'Доставка за 2 часа', 'Доставка ночью за 2 часа', 'Доставка за 1.5 часа', 'Больше продуктов за 2 часа'},
#     'Экспресс 30': {'Экспресс доставка', 'Доставка за 30 минут'},
#     'Экспресс 45': {'Экспресс за 45 минут', 'Доставка за 45 минут'},
#     'Экспресс 15': {'Экспресс за 15 минут', 'Доставка за 15 минут'},
#     'Доставка 1 час': {'Доставка за 1 час', 'Доставка за 1 час (с тоглом без слотов)', 'Доставка за 1 час от 2000р', 'Доставка за 1 час 99 руб'},
#     'Больше продуктов за 1 час': {'Больше продуктов за 1 час'},
#     'Доставка 3-4 часа': {'Доставка за 3 часа', 'Доставка за 3 часа (поселки)', 'Доставка за 3 часа (вместо 1 часа)', 'Доставка за 4 часа', 'Больше продуктов за 3 часа', 'Больше продуктов за 4 часа', 'Больше продуктов за 4 часа от 2000 руб', 'Доставка за 4 часа слоты, мин 8000р'},
#     'Вкусвильчик': {'ВкусВильчик'}
# }

# # Объединяем только нужные столбцы
# df = df.merge(service[['ID', 'Наименование']], left_on='id_service', right_on='ID', how='left')
# # Переименовываем столбцы
# df.rename(columns={'Наименование': 'service_name'}, inplace=True)
# # Проверяем, что 'service_name' — это Series, а не DataFrame
# if isinstance(df['service_name'], pd.DataFrame):
#     df['service_name'] = df['service_name'].iloc[:, 0]  # Берем первую колонку
# # Функция для присваивания групп
# def assign_group(service_name):
#     for group, services in service_groups.items():
#         if service_name in services:
#             return group
#     return "Другие"
# # Очистка строк и присвоение групп
# df['service_name'] = df['service_name'].astype(str).str.strip()
# df['group'] = df['service_name'].apply(assign_group)


### Подсчет смен курьеров

In [6]:
import numpy as np

# Преобразование 'id_courier' в int
df['id_courier'] = df['id_courier'].astype(int)

# Ко всем уникальным ади пулов добавляем начало и конец пула с учетом date_delivery_start и date_delivered
df['start_pool'] = df.groupby('id_pool')['date_delivery_start'].transform('max')
df['finished_pool'] = df.groupby('id_pool')['date_delivered'].transform('max')
df['first_order_pool']=df.groupby('id_pool')['date_delivery_start'].transform('min')

# Преобразование distance и time_delivery_order в float
df['gen_distance'] = df['gen_distance'].astype(str).str.replace(',', '.').replace('nan', None)
df['gen_distance'] = pd.to_numeric(df['gen_distance'], errors='coerce')
df['time_delivery_order'] = (df['finished_pool'] - df['start_pool']).dt.total_seconds() / 60

# Корректный расчет скорости
df['speed_order'] = df.apply(
    lambda row: round(row['gen_distance'] / (row['time_delivery_order'] / 60), 2)
    if pd.notna(row['time_delivery_order']) and row['time_delivery_order'] > 0
    else None,
    axis=1
)

# Группировка данных по курьерам
cour_pool_df = df[['id_courier', 'id_pool', 'date', 'start_pool', 'finished_pool', 'id_general', 'gen_distance', 'speed_order','courier_type', 'total_order_weight']].groupby(
    ['id_courier', 'id_pool', 'start_pool', 'finished_pool','courier_type']).agg(
        id_general_count=('id_general', 'count'),
        gen_distance=('gen_distance', 'mean'),
        speed_order=('speed_order', 'mean'),
        total_order_weight=('total_order_weight', 'sum')).reset_index()

# Добавляем столбец 'date' с использованием максимальной даты для каждой группы
cour_pool_df['date'] = cour_pool_df.groupby(['id_courier', 'id_pool'])['start_pool'].transform('max').dt.date

# Сортируем данные по курьеру и времени завершения пула
cour_pool_df = cour_pool_df.sort_values(by=['id_courier', 'finished_pool']).reset_index(drop=True)

# Создаем столбец prev_id_courier для проверки, какой курьер был в предыдущем пуле
cour_pool_df['prev_id_courier'] = cour_pool_df['id_courier'].shift(1)

# Для каждого курьера вычисляем время окончания предыдущего пула
cour_pool_df['prev_finished'] = cour_pool_df.groupby('id_courier')['finished_pool'].shift(1)

# Вместо преобразования в int, просто заменим NaN на -1 или оставим как float (если не требуется строго int)
cour_pool_df['prev_id_courier'] = cour_pool_df['prev_id_courier'].fillna(-1).astype(int)

# Теперь считаем gap_minutes, при этом проверяем, чтобы не была использована дата другого курьера
cour_pool_df['gap_minutes'] = np.where(
    (cour_pool_df['id_courier'] == cour_pool_df['prev_id_courier']),
    (cour_pool_df['start_pool'] - cour_pool_df['prev_finished']).dt.total_seconds() / 60,
    np.nan
)

# Отображаем данные
display(cour_pool_df.head(10))

# Пулы с отрицательным gap_minutes (одновременные пулы)
print('Отрицательные gap_minutes (одновременные пулы):')
display(cour_pool_df[cour_pool_df['gap_minutes'] < 0])

# Пулы идущие подряд (gap_minutes == 0)
print('Пулы идущие подряд:')
display(cour_pool_df[cour_pool_df['gap_minutes'] == 0])

# Курьер, у которого нет предыдущего пула
print('Новый курьер:')
new_courier_df = cour_pool_df[pd.isna(cour_pool_df['prev_finished']) & pd.isna(cour_pool_df['gap_minutes'])]
display(new_courier_df)

# Удаляем отрицательные gap_minutes (одновременные пулы)
cour_pool_df = cour_pool_df[cour_pool_df['gap_minutes'] >= 0].reset_index(drop=True)


,id_courier,id_pool,start_pool,finished_pool,courier_type,id_general_count,gen_distance,speed_order,total_order_weight,date,prev_id_courier,prev_finished,gap_minutes
0,0,60083561,2024-12-01 00:59:00,2024-12-01 01:25:00,Тип курьера неизвестен,1,1.662,3.84,7.253,2024-12-01,-1,NaT,NaN
1,0,60083578,2024-12-01 01:11:00,2024-12-01 01:51:00,Тип курьера неизвестен,1,3.564,5.35,1.453,2024-12-01,0,2024-12-01 01:25:00,-14.0
2,0,60083585,2024-12-01 02:44:00,2024-12-01 03:00:00,Тип курьера неизвестен,1,2.208,8.28,1.379,2024-12-01,0,2024-12-01 01:51:00,53.0
3,0,60083603,2024-12-01 02:53:00,2024-12-01 03:15:00,Тип курьера неизвестен,1,2.090,5.70,1.298,2024-12-01,0,2024-12-01 03:00:00,-7.0
4,0,60083611,2024-12-01 02:53:00,2024-12-01 03:18:00,Тип курьера неизвестен,1,2.518,6.04,3.123,2024-12-01,0,2024-12-01 03:15:00,-22.0
5,0,60083609,2024-12-01 03:11:00,2024-12-01 03:54:00,Тип курьера неизвестен,1,7.563,10.55,3.223,2024-12-01,0,2024-12-01 03:18:00,-7.0
6,0,60175355,2024-12-01 07:33:00,2024-12-01 08:15:00,Тип курьера неизвестен,1,3.849,5.50,1.413,2024-12-01,0,2024-12-01 03:54:00,219.0
7,0,60175324,2024-12-01 11:19:00,2024-12-01 12:14:00,Тип курьера неизвестен,1,2.926,3.19,16.305,2024-12-01,0,2024-12-01 08:15:00,184.0
8,0,60175347,2024-12-01 12:29:00,2024-12-01 13:00:00,Тип курьера неизвестен,1,4.024,7.79,9.702,2024-12-01,0,2024-12-01 12:14:00,15.0
9,0,60083596,2024-12-01 12:03:00,2024-12-01 13:45:00,Тип курьера неизвестен,1,4.541,2.67,2.106,2024-12-01,0,2024-12-01 13:00:00,-57.0


Отрицательные gap_minutes (одновременные пулы):


,id_courier,id_pool,start_pool,finished_pool,courier_type,id_general_count,gen_distance,speed_order,total_order_weight,date,prev_id_courier,prev_finished,gap_minutes
1,0,60083578,2024-12-01 01:11:00,2024-12-01 01:51:00,Тип курьера неизвестен,1,3.5640,5.350,1.453,2024-12-01,0,2024-12-01 01:25:00,-14.0
3,0,60083603,2024-12-01 02:53:00,2024-12-01 03:15:00,Тип курьера неизвестен,1,2.0900,5.700,1.298,2024-12-01,0,2024-12-01 03:00:00,-7.0
4,0,60083611,2024-12-01 02:53:00,2024-12-01 03:18:00,Тип курьера неизвестен,1,2.5180,6.040,3.123,2024-12-01,0,2024-12-01 03:15:00,-22.0
5,0,60083609,2024-12-01 03:11:00,2024-12-01 03:54:00,Тип курьера неизвестен,1,7.5630,10.550,3.223,2024-12-01,0,2024-12-01 03:18:00,-7.0
9,0,60083596,2024-12-01 12:03:00,2024-12-01 13:45:00,Тип курьера неизвестен,1,4.5410,2.670,2.106,2024-12-01,0,2024-12-01 13:00:00,-57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2723609,1799997073,66029396,2025-01-31 18:24:00,2025-01-31 18:55:00,Авто,4,3.1185,6.035,17.232,2025-01-31,1799997073,2025-01-31 18:55:00,-31.0
2723658,1799997097,66036069,2025-01-31 18:01:00,2025-01-31 19:43:00,Вело,1,1.7670,1.040,3.328,2025-01-31,1799997097,2025-01-31 18:20:00,-19.0
2723739,1799997328,66021383,2025-01-31 17:20:00,2025-01-31 18:01:00,Вело Свой,1,1.9940,2.920,10.891,2025-01-31,1799997328,2025-01-31 18:00:00,-40.0
2723740,1799997328,66023211,2025-01-31 17:31:00,2025-01-31 18:02:00,Вело Свой,5,1.1496,2.226,32.888,2025-01-31,1799997328,2025-01-31 18:01:00,-30.0


Пулы идущие подряд:


,id_courier,id_pool,start_pool,finished_pool,courier_type,id_general_count,gen_distance,speed_order,total_order_weight,date,prev_id_courier,prev_finished,gap_minutes
247,0,60951660,2024-12-09 20:43:00,2024-12-09 20:46:00,Тип курьера неизвестен,1,NaN,NaN,0.424,2024-12-09,0,2024-12-09 20:43:00,0.0
321,0,61273343,2024-12-12 03:03:00,2024-12-12 03:26:00,Тип курьера неизвестен,1,2.2910,5.980,2.071,2024-12-12,0,2024-12-12 03:03:00,0.0
475,0,61636757,2024-12-16 22:34:00,2024-12-16 23:03:00,Тип курьера неизвестен,1,2.4380,5.040,4.818,2024-12-16,0,2024-12-16 22:34:00,0.0
572,0,61950368,2024-12-19 22:12:00,2024-12-19 22:13:00,Тип курьера неизвестен,1,0.7960,47.760,0.548,2024-12-19,0,2024-12-19 22:12:00,0.0
586,0,62044375,2024-12-20 15:48:00,2024-12-20 16:06:00,Тип курьера неизвестен,1,3.6720,12.240,0.723,2024-12-20,0,2024-12-20 15:48:00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2722940,1799996538,65881905,2025-01-30 12:28:00,2025-01-30 12:50:00,Вело,1,1.7520,4.780,5.386,2025-01-30,1799996538,2025-01-30 12:28:00,0.0
2722960,1799996538,65995961,2025-01-31 13:05:00,2025-01-31 13:20:00,Вело,1,1.4870,5.950,1.183,2025-01-31,1799996538,2025-01-31 13:05:00,0.0
2723251,1799996744,65892874,2025-01-30 14:34:00,2025-01-30 14:44:00,Вело,2,2.9975,17.985,2.641,2025-01-30,1799996744,2025-01-30 14:34:00,0.0
2723601,1799997065,66040404,2025-01-31 19:49:00,2025-01-31 20:11:00,Авто,1,2.1060,5.740,8.375,2025-01-31,1799997065,2025-01-31 19:49:00,0.0


Новый курьер:


,id_courier,id_pool,start_pool,finished_pool,courier_type,id_general_count,gen_distance,speed_order,total_order_weight,date,prev_id_courier,prev_finished,gap_minutes
0,0,60083561,2024-12-01 00:59:00,2024-12-01 01:25:00,Тип курьера неизвестен,1,1.662000,3.840000,7.253,2024-12-01,-1,NaT,NaN
1953,65709895,60783534,2024-12-08 10:21:00,2024-12-08 10:28:00,Авто,1,0.440000,3.770000,0.833,2024-12-08,0,NaT,NaN
1959,94477626,60468617,2024-12-05 05:46:00,2024-12-05 06:18:00,Авто,4,2.391750,4.485000,7.282,2024-12-05,65709895,NaT,NaN
2345,135044252,65042883,2025-01-22 11:31:00,2025-01-22 12:18:00,Авто,4,4.228500,5.400000,13.023,2025-01-22,94477626,NaT,NaN
2355,154799142,60093224,2024-12-01 09:39:00,2024-12-01 10:08:00,Авто,5,1.774400,3.672000,30.370,2024-12-01,135044252,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2723735,1799997328,66007169,2025-01-31 14:44:00,2025-01-31 14:56:00,Вело Свой,2,0.841000,4.210000,11.396,2025-01-31,1799997320,NaT,NaN
2723748,1799997341,66027896,2025-01-31 18:02:00,2025-01-31 18:29:00,Вело,2,2.168500,4.815000,13.156,2025-01-31,1799997328,NaT,NaN
2723750,1799997355,66018343,2025-01-31 16:03:00,2025-01-31 17:51:00,Авто,7,9.908571,5.504286,17.979,2025-01-31,1799997341,NaT,NaN
2723752,1799997359,66020384,2025-01-31 16:26:00,2025-01-31 17:32:00,Вело,2,3.047000,2.770000,8.989,2025-01-31,1799997355,NaT,NaN


Посчитаем кол-во смен и их длительность:

In [ ]:
# --- ЛОГИКА ПОДСЧЁТА СМЕН ---

# Добавляем флаг новой смены: если разница между пулами > 2.5 часов (150 минут), то это новая смена
cour_pool_df['new_shift_flag'] = (cour_pool_df['gap_minutes'].isna()) | (cour_pool_df['gap_minutes'] > 150)
# Присваиваем каждой записи номер смены (кумулятивная сумма флага)
cour_pool_df['shift_num'] = cour_pool_df.groupby('id_courier')['new_shift_flag'].cumsum()

# Сколько минут везся один пул 
cour_pool_df['delivery_duration_minutes'] = (cour_pool_df['finished_pool'] - cour_pool_df['start_pool']).dt.total_seconds() / 60

cour_pool_df

,id_courier,id_pool,start_pool,finished_pool,courier_type,id_general_count,gen_distance,speed_order,total_order_weight,date,prev_id_courier,prev_finished,gap_minutes,new_shift_flag,shift_num,delivery_duration_minutes
0,0,60083585,2024-12-01 02:44:00,2024-12-01 03:00:00,Тип курьера неизвестен,1,2.208000,8.280000,1.379,2024-12-01,0,2024-12-01 01:51:00,53.0,False,0,16.0
1,0,60175355,2024-12-01 07:33:00,2024-12-01 08:15:00,Тип курьера неизвестен,1,3.849000,5.500000,1.413,2024-12-01,0,2024-12-01 03:54:00,219.0,True,1,42.0
2,0,60175324,2024-12-01 11:19:00,2024-12-01 12:14:00,Тип курьера неизвестен,1,2.926000,3.190000,16.305,2024-12-01,0,2024-12-01 08:15:00,184.0,True,2,55.0
3,0,60175347,2024-12-01 12:29:00,2024-12-01 13:00:00,Тип курьера неизвестен,1,4.024000,7.790000,9.702,2024-12-01,0,2024-12-01 12:14:00,15.0,False,2,31.0
4,0,60175337,2024-12-01 16:14:00,2024-12-01 17:03:00,Тип курьера неизвестен,1,3.448000,4.220000,1.033,2024-12-01,0,2024-12-01 13:45:00,149.0,False,2,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266838,1799997328,66042516,2025-01-31 20:27:00,2025-01-31 20:44:00,Вело Свой,3,1.075333,3.796667,4.788,2025-01-31,1799997328,2025-01-31 20:19:00,8.0,False,0,17.0
2266839,1799997328,66046687,2025-01-31 21:11:00,2025-01-31 21:33:00,Вело Свой,5,0.997000,2.718000,19.969,2025-01-31,1799997328,2025-01-31 20:44:00,27.0,False,0,22.0
2266840,1799997341,66038168,2025-01-31 19:18:00,2025-01-31 19:54:00,Вело,1,2.562000,4.270000,0.893,2025-01-31,1799997341,2025-01-31 18:29:00,49.0,False,0,36.0
2266841,1799997355,66039118,2025-01-31 19:33:00,2025-01-31 20:54:00,Авто,8,10.450375,7.741250,49.696,2025-01-31,1799997355,2025-01-31 17:51:00,102.0,False,0,81.0


In [8]:
# Заменяем значения в 'gap_minutes' больше 150 на NaN
cour_pool_df['gap_minutes'] = cour_pool_df['gap_minutes'].apply(lambda x: np.nan if x > 150 else x)

# Теперь агрегируем данные для подсчета статистик по сменам
shift_summary = cour_pool_df.groupby(['id_courier', 'shift_num','courier_type']).agg(
    shift_start=('start_pool', 'min'),  # Начало смены (первый пул)
    shift_end=('finished_pool', 'max'),  # Конец смены (последний пул)
    total_delivery_time_minutes=('delivery_duration_minutes', 'sum'),  # Сумма времени доставки всех пулов
    mean_delivery_time_pool=('delivery_duration_minutes', 'mean'),  # Среднее время доставки пула
    pool_count=('id_pool', 'count'),  # Количество пулов в смене
    order_count=('id_general_count', 'sum'),  # Сумма количества заказов
    gap_minutes=('gap_minutes', 'mean'),  # Среднее время разрыва между пулов
    gen_distance=('gen_distance', 'mean'),
    speed_order=('speed_order', 'mean'),
    order_weight=('total_order_weight', 'mean')
).reset_index()


# Вычисляем общую длительность смены (разница между началом первого и окончанием последнего пула)
shift_summary['shift_duration_minutes'] = (shift_summary['shift_end'] - shift_summary['shift_start']).dt.total_seconds() / 3600
shift_summary['total_delivery_time_minutes']=shift_summary['total_delivery_time_minutes']/60
# display(shift_summary['id_courier'].unique())
shift_summary

,id_courier,shift_num,courier_type,shift_start,shift_end,total_delivery_time_minutes,mean_delivery_time_pool,pool_count,order_count,gap_minutes,gen_distance,speed_order,order_weight,shift_duration_minutes
0,0,0,Тип курьера неизвестен,2024-12-01 02:44:00,2024-12-01 03:00:00,0.266667,16.000000,1,1,53.000000,2.208000,8.280000,1.379000,0.266667
1,0,1,Тип курьера неизвестен,2024-12-01 07:33:00,2024-12-01 08:15:00,0.700000,42.000000,1,1,NaN,3.849000,5.500000,1.413000,0.700000
2,0,2,Тип курьера неизвестен,2024-12-01 11:19:00,2024-12-02 03:15:00,5.750000,31.363636,11,11,30.800000,2.703091,5.339091,6.944273,15.933333
3,0,3,Тип курьера неизвестен,2024-12-02 10:21:00,2024-12-03 13:18:00,7.733333,21.090909,22,22,42.714286,2.079409,11.797273,6.707864,26.950000
4,0,4,Тип курьера неизвестен,2024-12-03 16:20:00,2024-12-04 02:32:00,3.716667,18.583333,12,12,29.636364,2.822417,19.695833,1.892417,10.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341780,1799997320,0,Вело,2025-01-31 19:31:00,2025-01-31 23:20:00,2.100000,25.200000,5,10,24.000000,3.323900,8.618000,12.947800,3.816667
341781,1799997328,0,Вело Свой,2025-01-31 15:22:00,2025-01-31 21:33:00,2.916667,19.444444,9,28,23.888889,1.232028,3.919685,12.277667,6.183333
341782,1799997341,0,Вело,2025-01-31 19:18:00,2025-01-31 19:54:00,0.600000,36.000000,1,1,49.000000,2.562000,4.270000,0.893000,0.600000
341783,1799997355,0,Авто,2025-01-31 19:33:00,2025-01-31 20:54:00,1.350000,81.000000,1,8,102.000000,10.450375,7.741250,49.696000,1.350000


### Разбиваем на 15-минутные блоки и подсчитываем статистики и определяем статусы курьеров

Разбиваем на 15-мин блоки и считаем колво заказов и пулов в тот день 

1. Сортировка по delivery_start
2. gap_minutes - это время возврата + время доставки заказа, так как ты разницу считаешь между стартами

In [ ]:
# 1. Создаём 15-минутные интервалы
min_date = df['date_delivery_start'].min().floor('15min')
max_date = df['date_delivery_start'].max().ceil('15min')
date_range = pd.date_range(start=min_date, end=max_date, freq='15min')

# 2. Округляем время доставки до 15-минутного интервала
df['date_group'] = df['first_order_pool'].dt.floor('15min')

# 3. Создаём каркас данных: все комбинации курьеров и временных интервалов
couriers = df['id_courier'].unique()
grid = pd.MultiIndex.from_product([couriers, date_range], names=['id_courier', 'date_group'])
courier_df = pd.DataFrame(index=grid).reset_index()

# 4. Определяем смены (разрыв между заказами > 2.5 часа)
df = df.sort_values(['id_courier', 'date_delivery_start'])
df['gap_minutes'] = df.groupby('id_courier')['date_delivery_start'].diff().dt.total_seconds() / 60 # время возврата + доставка 
df['new_shift_flag'] = (df['gap_minutes'].isna()) | (df['gap_minutes'] > 150)
df['shift_num'] = df.groupby('id_courier')['new_shift_flag'].cumsum()

# 5. Привязываем смены к 15-минутным интервалам
courier_df = courier_df.merge(df[['id_courier', 'date_group', 'shift_num']],
                              on=['id_courier', 'date_group'], how='left')
courier_df['shift_num'] = courier_df.groupby('id_courier')['shift_num'].ffill()

# 6. Cumulative count of unique `id_pool` in each shift
df_sorted = df.sort_values(['id_courier', 'shift_num', 'date_delivery_start'])

# Drop duplicates before applying cumulative count
df_sorted = df_sorted.drop_duplicates(subset=['id_courier', 'shift_num', 'id_pool'])

df_sorted['cumulative_pools'] = df_sorted.groupby(['id_courier', 'shift_num'])['id_pool'].transform('cumcount') + 1

# Get the max number of pools per time interval
unique_pools_per_interval = (
    df_sorted.groupby(['id_courier', 'shift_num', 'date_group'])['cumulative_pools']
    .max()
    .reset_index()
    .rename(columns={'cumulative_pools': 'pools_count'})
)

courier_df = courier_df.merge(unique_pools_per_interval, on=['id_courier', 'shift_num', 'date_group'], how='left')
courier_df['pools_count'] = courier_df.groupby(['id_courier', 'shift_num'])['pools_count'].ffill().fillna(0).astype(int)

# 7. Кумулятивный подсчёт уникальных заказов (id_general)
df['cumulative_orders'] = (
    df.drop_duplicates(subset=['id_courier', 'shift_num', 'id_general'])
    .groupby(['id_courier', 'shift_num']).cumcount() + 1
)

unique_orders_per_interval = (
    df.groupby(['id_courier', 'shift_num', 'date_group'])['cumulative_orders']
    .max()
    .reset_index()
    .rename(columns={'cumulative_orders': 'order_count'})
)

courier_df = courier_df.merge(unique_orders_per_interval, on=['id_courier', 'shift_num', 'date_group'], how='left')
courier_df['order_count'] = courier_df.groupby(['id_courier', 'shift_num'])['order_count'].ffill().fillna(0).astype(int)

# 8. Подсчёт количества смен за день
courier_df['date'] = courier_df['date_group'].dt.date
shift_count_per_day = courier_df.groupby(['id_courier', 'date'])['shift_num'].nunique().reset_index()
shift_count_per_day.rename(columns={'shift_num': 'shift_count_per_day'}, inplace=True)
courier_df = courier_df.merge(shift_count_per_day, on=['id_courier', 'date'], how='left')
courier_df['shift_count_per_day'] = courier_df['shift_count_per_day'].fillna(0).astype(int)

Определяем статус курьера: 

**Dataframes которые есть:**
df:
1. start_pool - время начала доставки пула
2. finished_pool - время когда курьер закончил доставлять пул
3. time_delivery_order - сколько времени доставлялся пул
4. gap_minutes - сколько минут до следующего пула доставления пула (вместе со временм когда курьер может вернуться в ТТ и выбирать заказ)
5. date_status -время когда заказ стал доступен для выбора курьерм (НЕ НУЖНО)
6. date_delivery_start  - время, когда курьер выбрал заказ для доставки
7. first_order_pool- время когда самый первый  заказ стал доступен для выбора курьерм (для всех заказов в пуле стоит самый первый)
8. date_group - 15-ти минутные диапозоны пример ( если start_pool= 6:29, то заказ относиться к группе заказов с 6:15 )

shift_summary - статистики расчитанные для заказов
courier_df - курьеры с диапозонами и подсчитанными сменами 


**Обозначение статусов**

"Доставляет" — Курьер активно доставляет заказ.

"Не работает" — Курьер не активен, не доставляет и не ожидает заказа (может быть в процессе возвращения в ТТ или в перерыве).

"Готов взять заказ" - Курьер вернулся в ТТ, но еще не начал доставлять заказ (выбирает заказы).

**Статус "Доставлет":** Этот статус будет означать, что курьер в данный момент доставляет заказ или находится в процессе завершения последнего заказа. Этот статус следует присвоить курьеру, если:

    1. Он находится в промежутке между временем завершения предыдущего заказа и временем доставки.
    2. Он активно работает над текущим заказом или передвигается к следующему заказу.
    3. В этот момент курьер не может взять новый заказ, так как он еще завершает текущую доставку.

**Статус "Не работает":** Этот статус будет присвоен в периоды, когда курьер не активен вообще. Это может означать, что:

    1. Курьер не находится в процессе доставки или не вернулся в ТТ. Разница между last_delivered и следующим заказом (next_first_chosen) больше, чем 150 минут
    2. Он не активен в течение значительного времени, например, если он ждет нового заказа, но его текущий заказ еще не был выбран.

**Статус "Готов взять заказ":** Этот статус будет присвоен в периоды, когда курьер вернулся в ТТ, но еще не начал доставлять заказ. Это может означать, что:

    1. Курьер не находится в процессе доставки, но уже вернулся в ТТ.
    2. Курьер может взять заказ и начать доставку.

1. сортируем по айди курьеров , пулов и date_group
2. df['Доставляет'], если date_group between start_pool И finished_pool и не назначен статус Готов взять заказ
3. df['Не работает'] если его gap_minutes если больше 150  значит он не работал весь период с finished_pool предыдущего  по first_order_pool след пула
4. df['Готов взять заказ'] если  gap_minutes меньше 150 минут значит он готов уже брать заказы с весь период с finished_pool предыдущего  по first_order_pool след пула

**Заполнение пропусков**:
Предположим, что есть DataFrame all_df, где некоторые строки уже имеют статус (например, 'Доставляет', 'Не работает', 'Готов взять заказ'), а некоторые строки — пропущенные (NaN).

Также есть столбцы:
  - id_courier
  - date_group       (временная метка 15-минутного шага)
  - finished_pool    (время окончания предыдущего пула, может быть NaN в "промежуточных" строках)
  - gap_minutes      (рассчитанное время до следующего пула, может быть NaN)
  - status           (текущий статус или NaN)

Логика, которую нужно применить для заполнения пропусков:
 1) Если date_group <= finished_pool (предыдущего пула) => "Доставляет".
 2) Иначе, если gap_minutes (следующего пула) > 150 => "Не работает".
 3) Иначе => "Готов взять заказ".

При этом "предыдущий пул" и "следующий пул" мы берём через ffill/bfill:
  - "предыдущий пул" → last known finished_pool (по группе id_courier)
  - "следующий пул" → first known gap_minutes (по группе id_courier)
чтобы каждая строка имела контекст о том, когда закончился прошлый пул
и сколько минут до следующего.


In [ ]:
df = df.sort_values(by=['id_courier', 'id_pool', 'date_group'])

df['status'] = 'Готов взять заказ'
courier_status_df = df[['id_courier','id_pool','id_general','date' ,'courier_type','N','date_group', 'status', 'gap_minutes', 'start_pool', 'finished_pool', 'first_order_pool']]
ex_courier_status_df=courier_status_df[(courier_status_df['id_courier']==94477626)&(courier_status_df['date_group']>='2024-01-05 7:15:00')&(courier_status_df['date_group']<='2024-01-05 10:15:00')& (courier_status_df['id_pool']==33307153)]

date_range = pd.date_range(start=min_date, end=max_date, freq='15min')

# Собираем все комбинации id_courier × date_group
couriers = df['id_courier'].unique()
grid = pd.MultiIndex.from_product([couriers, date_range], names=['id_courier', 'date_group'])
shift_df = pd.DataFrame(index=grid).reset_index()
shift_df.sort_values(['id_courier', 'date_group'])
courier_status_df.sort_values(['id_courier', 'date_group'])

shift_df_indexed = shift_df.set_index(['id_courier', 'date_group'])

# Аналогично делаем в courier_status_df.
courier_status_indexed = courier_status_df.set_index(['id_courier', 'date_group'])

# Выполняем left-join (по умолчанию), чтобы сохранить все строки из shift_df.
all_df = shift_df_indexed.join(
    courier_status_indexed[
        ['id_pool' ,'date', 'courier_type', 'N', 'status', 'gap_minutes', 'start_pool', 'finished_pool', 'first_order_pool']
    ],
    how='left',
    lsuffix='',
    rsuffix='_r'
)

# Возвращаем индекс в столбцы, если нужно
all_df = all_df.reset_index()
all_df.sort_values(['id_courier', 'date_group', 'id_pool'])
all_df = all_df.drop_duplicates(subset=['id_courier', 'date_group', 'id_pool'])

# 1. Сортируем DataFrame по курьеру и времени
all_df = all_df.sort_values(['id_courier', 'date_group'])

# 2. Для каждой группы (курьера) «протягиваем» (forward fill) finished_pool,
#    чтобы каждая строка знала, когда закончился предыдущий пул.
all_df['prev_finished_pool'] = all_df.groupby('id_courier')['finished_pool'].ffill()

# 3. Для gap_minutes (следующего пула) делаем backward fill,
#    чтобы каждая строка знала, сколько времени до ближайшего (следующего) пула.
all_df['next_gap_minutes'] = all_df.groupby('id_courier')['gap_minutes'].bfill()

# 4. Создаём маску для строк, где статус ещё не заполнен (NaN).
mask_null_status = all_df['status'].isna()

# 5. Определяем три ключевых условия:

# a) «Доставляет»: если date_group <= prev_finished_pool и next_gap_minutes < 150
mask_deliver = (
    mask_null_status &
    (all_df['date_group'] <= all_df['prev_finished_pool']) &
    (all_df['next_gap_minutes'] < 150)
)

# b) «Не работает»: если date_group > prev_finished_pool и next_gap_minutes > 150
mask_not_working = (
    mask_null_status &
    (all_df['date_group'] > all_df['prev_finished_pool']) &
    (all_df['next_gap_minutes'] > 150)
)

# c) «Готов взять заказ»: если date_group > prev_finished_pool и next_gap_minutes <= 150
mask_ready = (
    mask_null_status &
    (all_df['date_group'] > all_df['prev_finished_pool']) &
    (all_df['next_gap_minutes'] <= 150)
)

# 6. Применяем маски к столбцу status
all_df.loc[mask_deliver, 'status'] = 'Доставляет'
all_df.loc[mask_not_working, 'status'] = 'Не работает'
all_df.loc[mask_ready, 'status'] = 'Готов взять заказ'

# 7. Оставшиеся пропуски (если вдруг что-то не попало в три условия) заполним «Не работает»
all_df['status'] = all_df['status'].fillna('Не работает')

# 8. Удаляем временные столбцы
all_df.drop(columns=['prev_finished_pool', 'next_gap_minutes', 'date'], inplace=True)
all_df = all_df[all_df['id_courier'] > 0]
all_df

In [ ]:
# соединяем с инфо о колве пулов и заказов в тот день
all_df = all_df.merge(courier_df[['id_courier', 'date_group', 'pools_count', 'order_count', 'shift_num', 'shift_count_per_day']], on=['id_courier', 'date_group'], how='left')
all_df.drop_duplicates(inplace=True)
all_df

,id_courier,date_group,id_pool,courier_type,N,status,gap_minutes,start_pool,finished_pool,first_order_pool,pools_count,order_count,shift_num,shift_count_per_day
0,65709895,2024-12-01 00:00:00,NaN,NaN,NaN,Не работает,NaN,NaT,NaT,NaT,0,0,NaN,0
1,65709895,2024-12-01 00:15:00,NaN,NaN,NaN,Не работает,NaN,NaT,NaT,NaT,0,0,NaN,0
2,65709895,2024-12-01 00:30:00,NaN,NaN,NaN,Не работает,NaN,NaT,NaT,NaT,0,0,NaN,0
3,65709895,2024-12-01 00:45:00,NaN,NaN,NaN,Не работает,NaN,NaT,NaT,NaT,0,0,NaN,0
4,65709895,2024-12-01 01:00:00,NaN,NaN,NaN,Не работает,NaN,NaT,NaT,NaT,0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81316877,1799997361,2025-02-01 18:45:00,NaN,NaN,NaN,Не работает,NaN,NaT,NaT,NaT,2,6,1.0,1
81316878,1799997361,2025-02-01 19:00:00,NaN,NaN,NaN,Не работает,NaN,NaT,NaT,NaT,2,6,1.0,1
81316879,1799997361,2025-02-01 19:15:00,NaN,NaN,NaN,Не работает,NaN,NaT,NaT,NaT,2,6,1.0,1
81316880,1799997361,2025-02-01 19:30:00,NaN,NaN,NaN,Не работает,NaN,NaT,NaT,NaT,2,6,1.0,1


### Высчитываем исторические статистики курьеров

In [15]:
shift_summary['date'] = shift_summary['shift_start'].dt.date
shift_summary = shift_summary[shift_summary['id_courier'] > 0]
shift_summary

,id_courier,shift_num,courier_type,shift_start,shift_end,total_delivery_time_minutes,mean_delivery_time_pool,pool_count,order_count,gap_minutes,gen_distance,speed_order,order_weight,shift_duration_minutes,date
94,65709895,1,Авто,2024-12-12 09:11:00,2024-12-12 09:19:00,0.133333,8.000000,1,1,NaN,0.295000,2.210000,1.018000,0.133333,2024-12-12
95,65709895,2,Авто,2024-12-16 09:05:00,2024-12-16 09:10:00,0.083333,5.000000,1,1,NaN,1.838000,22.060000,1.578000,0.083333,2024-12-16
96,65709895,3,Авто,2025-01-13 09:39:00,2025-01-13 09:45:00,0.100000,6.000000,1,1,NaN,0.294000,2.940000,1.618000,0.100000,2025-01-13
97,65709895,4,Авто,2025-01-16 15:31:00,2025-01-16 15:38:00,0.116667,7.000000,1,1,NaN,0.294000,2.520000,0.463000,0.116667,2025-01-16
98,65709895,5,Авто,2025-01-23 09:42:00,2025-01-23 09:51:00,0.150000,9.000000,1,1,NaN,0.294000,1.960000,0.618000,0.150000,2025-01-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341780,1799997320,0,Вело,2025-01-31 19:31:00,2025-01-31 23:20:00,2.100000,25.200000,5,10,24.000000,3.323900,8.618000,12.947800,3.816667,2025-01-31
341781,1799997328,0,Вело Свой,2025-01-31 15:22:00,2025-01-31 21:33:00,2.916667,19.444444,9,28,23.888889,1.232028,3.919685,12.277667,6.183333,2025-01-31
341782,1799997341,0,Вело,2025-01-31 19:18:00,2025-01-31 19:54:00,0.600000,36.000000,1,1,49.000000,2.562000,4.270000,0.893000,0.600000,2025-01-31
341783,1799997355,0,Авто,2025-01-31 19:33:00,2025-01-31 20:54:00,1.350000,81.000000,1,8,102.000000,10.450375,7.741250,49.696000,1.350000,2025-01-31


In [16]:
shift_df=shift_summary.groupby(['id_courier', 'date']).agg({
    'total_delivery_time_minutes': 'sum',
	'pool_count' : 'sum',
    'shift_duration_minutes': 'sum',
    'gap_minutes':'mean',
    'shift_num': 'nunique',
    'order_count': 'sum',
    'order_weight': 'mean'
}).rename(columns={
   'total_delivery_time_minutes': 'delivery_time_shift',
   'pool_count': 'count_pool_shift',
   'shift_duration_minutes': 'time_shift',
   'gap_minutes': 'returned_time',
   'shift_num': 'count_shift',
   'order_count': 'order_counts'}).reset_index()
shift_df

,id_courier,date,delivery_time_shift,count_pool_shift,time_shift,returned_time,count_shift,order_counts,order_weight
0,65709895,2024-12-12,0.133333,1,0.133333,NaN,1,1,1.018000
1,65709895,2024-12-16,0.083333,1,0.083333,NaN,1,1,1.578000
2,65709895,2025-01-13,0.100000,1,0.100000,NaN,1,1,1.618000
3,65709895,2025-01-16,0.116667,1,0.116667,NaN,1,1,0.463000
4,65709895,2025-01-23,0.150000,1,0.150000,NaN,1,1,0.618000
...,...,...,...,...,...,...,...,...,...
288310,1799997320,2025-01-31,2.100000,5,3.816667,24.000000,1,10,12.947800
288311,1799997328,2025-01-31,2.916667,9,6.183333,23.888889,1,28,12.277667
288312,1799997341,2025-01-31,0.600000,1,0.600000,49.000000,1,1,0.893000
288313,1799997355,2025-01-31,1.350000,1,1.350000,102.000000,1,8,49.696000


In [17]:
shift_df = shift_df.groupby(['id_courier']).agg({
    'delivery_time_shift': 'mean',
 'count_pool_shift' : 'mean',
    'time_shift': 'mean',
    'returned_time':'mean',
    'count_shift': 'mean',
    'order_counts': 'mean',
    'order_weight': 'mean'
}).reset_index()
shift_df

,id_courier,delivery_time_shift,count_pool_shift,time_shift,returned_time,count_shift,order_counts,order_weight
0,65709895,0.116667,1.000000,0.116667,NaN,1.000000,1.000000,1.059000
1,94477626,4.971839,11.172414,13.210345,48.132047,1.068966,50.551724,18.354300
2,135044252,3.900000,4.500000,6.258333,38.250000,1.000000,14.000000,15.972800
3,154799142,0.400000,1.000000,0.400000,106.000000,1.000000,3.000000,21.546000
4,165691443,2.023611,8.833333,5.425000,37.419264,1.250000,13.916667,5.860119
...,...,...,...,...,...,...,...,...
11297,1799997320,2.100000,5.000000,3.816667,24.000000,1.000000,10.000000,12.947800
11298,1799997328,2.916667,9.000000,6.183333,23.888889,1.000000,28.000000,12.277667
11299,1799997341,0.600000,1.000000,0.600000,49.000000,1.000000,1.000000,0.893000
11300,1799997355,1.350000,1.000000,1.350000,102.000000,1.000000,8.000000,49.696000


### Соединяем статистики с 15-ти минутными промежутками

In [ ]:
all_df = all_df.merge(shift_df, on=['id_courier'], how='left')

In [19]:
all_df.drop(columns=['gap_minutes', 'start_pool', 'finished_pool', 'first_order_pool'], inplace=True)
all_df

,id_courier,date_group,id_pool,courier_type,N,status,pools_count,order_count,shift_num,shift_count_per_day,delivery_time_shift,count_pool_shift,time_shift,returned_time,count_shift,order_counts,order_weight
0,65709895,2024-12-01 00:00:00,NaN,NaN,NaN,Не работает,0,0,NaN,0,0.116667,1.0,0.116667,NaN,1.0,1.0,1.059
1,65709895,2024-12-01 00:15:00,NaN,NaN,NaN,Не работает,0,0,NaN,0,0.116667,1.0,0.116667,NaN,1.0,1.0,1.059
2,65709895,2024-12-01 00:30:00,NaN,NaN,NaN,Не работает,0,0,NaN,0,0.116667,1.0,0.116667,NaN,1.0,1.0,1.059
3,65709895,2024-12-01 00:45:00,NaN,NaN,NaN,Не работает,0,0,NaN,0,0.116667,1.0,0.116667,NaN,1.0,1.0,1.059
4,65709895,2024-12-01 01:00:00,NaN,NaN,NaN,Не работает,0,0,NaN,0,0.116667,1.0,0.116667,NaN,1.0,1.0,1.059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70962465,1799997361,2025-02-01 18:45:00,NaN,NaN,NaN,Не работает,2,6,1.0,1,0.616667,1.0,0.616667,44.0,1.0,3.0,11.773
70962466,1799997361,2025-02-01 19:00:00,NaN,NaN,NaN,Не работает,2,6,1.0,1,0.616667,1.0,0.616667,44.0,1.0,3.0,11.773
70962467,1799997361,2025-02-01 19:15:00,NaN,NaN,NaN,Не работает,2,6,1.0,1,0.616667,1.0,0.616667,44.0,1.0,3.0,11.773
70962468,1799997361,2025-02-01 19:30:00,NaN,NaN,NaN,Не работает,2,6,1.0,1,0.616667,1.0,0.616667,44.0,1.0,3.0,11.773


In [20]:
all_df.sort_values(["id_courier",'date_group'], inplace=True)

In [21]:
all_df.to_csv('cour_12-01.csv', index=False)